In [15]:
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np


In [18]:
class ResNet50MultiLabel(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, n_classes)

    def forward(self, x):
        return self.model(x)


In [ ]:
transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()
])


In [ ]:
from google.colab import files
uploaded = files.upload()

!unzip best_resnet50.zip

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_classes = 14
state_dict = torch.load("best_resnet50.pth", map_location=device)
model = ResNet50MultiLabel(n_classes)
model.load_state_dict(state_dict)
model.eval()


Saving best_resnet50.pth to best_resnet50.pth
Archive:  best_resnet50.zip
replace best_resnet50.pth? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


ResNet50MultiLabel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
 

In [25]:
def predict_xray(path_to_image):
    # Load and transform
    image = Image.open(path_to_image).convert("RGB")
    image_np = np.array(image)
    image_tensor = transform(image=image_np)["image"].unsqueeze(0)

    # Forward pass
    with torch.no_grad():
        logits = model(image_tensor)
        probs = torch.sigmoid(logits).squeeze().numpy()

    return probs


In [26]:
probs = predict_xray("/content/new_xray.png")
probs


array([0.3631686 , 0.62528783, 0.24734783, 0.21874443, 0.04872143,
       0.05562816, 0.02812475, 0.03991663, 0.05133834, 0.03115132,
       0.02569919, 0.04451052, 0.06164244, 0.0144746 ], dtype=float32)

In [27]:
CLASS_NAMES = [
    "Atelectasis", "Cardiomegaly", "Effusion", "Infiltration",
    "Mass", "Nodule", "Pneumonia", "Pneumothorax",
    "Consolidation", "Edema", "Emphysema", "Fibrosis",
    "Pleural_Thickening", "Hernia"
]

for cls, p in zip(CLASS_NAMES, probs):
    print(f"{cls:20s} : {p:.3f}")


Atelectasis          : 0.363
Cardiomegaly         : 0.625
Effusion             : 0.247
Infiltration         : 0.219
Mass                 : 0.049
Nodule               : 0.056
Pneumonia            : 0.028
Pneumothorax         : 0.040
Consolidation        : 0.051
Edema                : 0.031
Emphysema            : 0.026
Fibrosis             : 0.045
Pleural_Thickening   : 0.062
Hernia               : 0.014


In [28]:
predicted_labels = [CLASS_NAMES[i] for i, p in enumerate(probs) if p > 0.5]
predicted_labels


['Cardiomegaly']